In [1]:
import pickle
import pandas as pd
import numpy as np
import pickle
# data_root_dir : /ts/projects/mimic4_cxr/tabular/preprocess/processed_data/mimic4/

In [5]:
task = 'mortality'
target_days = 90

# cp -r /ts/projects/mimic4_cxr/tabular/preprocess/processed_data/mimic4/ /home/data/2025_MIMICIV_processed/
hosp_adm_dict = pickle.load(open(f"/home/data/2025_MIMICIV_processed/mimic4/hosp_adm_dict_{target_days}days.pkl", "rb"))

# train ids path : /home/data/2025_MIMICIV_processed/mimic4/task:mortality_30days/train_admission_ids.txt
train_ids = pd.read_csv(f"/home/data/2025_MIMICIV_processed/mimic4/task:{task}_{target_days}days/train_admission_ids.txt", header=None)[0].values
val_ids = pd.read_csv(f"/home/data/2025_MIMICIV_processed/mimic4/task:{task}_{target_days}days/val_admission_ids.txt", header=None)[0].values
test_ids = pd.read_csv(f"/home/data/2025_MIMICIV_processed/mimic4/task:{task}_{target_days}days/test_admission_ids.txt", header=None)[0].values

In [6]:
train_tabular_ids, train_note_ids = [], []
val_tabular_ids, val_note_ids = [], []
test_tabular_ids, test_note_ids = [], []

for key, items in hosp_adm_dict.items() : 
    # items : HOSP_ADM ID-22595853 (1132.0 min): mortality-False, readmission-False
    mortality = items.mortality
    key = int(key)
    # skip if mortality is None
    if mortality is None :
        continue
    
    # Use all data for tabular
    # skip if discharge note is None
    if items.discharge is None :
        train_tabular_ids.append(key) if key in train_ids else \
        val_tabular_ids.append(key) if key in val_ids else \
        test_tabular_ids.append(key) if key in test_ids else None
        continue
    
    train_note_ids.append(key) if key in train_ids else \
    val_note_ids.append(key) if key in val_ids else \
    test_note_ids.append(key) if key in test_ids else None
    
    train_tabular_ids.append(key) if key in train_ids else \
    val_tabular_ids.append(key) if key in val_ids else \
    test_tabular_ids.append(key) if key in test_ids else None

print(f"Original Train : {len(train_ids)}, Val : {len(val_ids)}, Test : {len(test_ids)}")
print(f"Tabular Train : {len(train_tabular_ids)}, Val : {len(val_tabular_ids)}, Test : {len(test_tabular_ids)}")
print(f"Note Train : {len(train_note_ids)}, Val : {len(val_note_ids)}, Test : {len(test_note_ids)}")


# Original Train : 126178, Val : 18026, Test : 36051
# Tabular Train : 126178, Val : 18026, Test : 36051
# Note Train : 81656, Val : 11762, Test : 23414

Original Train : 109321, Val : 15617, Test : 31235
Tabular Train : 109321, Val : 15617, Test : 31235
Note Train : 71113, Val : 10050, Test : 20215


In [7]:
tabular = {
    'train' : train_tabular_ids,
    'val' : val_tabular_ids,
    'test' : test_tabular_ids
}
note = {
    'train' : train_note_ids,
    'val' : val_note_ids,
    'test' : test_note_ids
}


with open(f"/home/data/2025_MIMICIV_processed/mimic4/task:{task}_{target_days}days/SSL_tabular_ids.pkl", "wb") as f :
    pickle.dump(tabular, f)
with open(f"/home/data/2025_MIMICIV_processed/mimic4/task:{task}_{target_days}days/SSL_note_ids.pkl", "wb") as f :
    pickle.dump(note, f)